### Converting the dataset.xlsx into datalog facts

In [ ]:
import pandas as pd
import os

# Données Patients
patients_df = pd.read_excel("../data/dataset.xlsx", sheet_name="data")

# Création des fichiers Observed.facts et History.facts s'ils n'existent pas
observed_file_path = "../data/Observed.facts"
history_file_path = "../data/History.facts"

if not os.path.exists(observed_file_path) and not os.path.exists(history_file_path):
    with open(observed_file_path, "w") as observed_file, open(history_file_path, "w") as history_file:
        for _, row in patients_df.iterrows():
            patient_id = row["PatientID"]
            
            # Écrire dans le fichier Observed.facts si le symptôme observé est présent
            if pd.notna(row["Observed_Symptom"]):
                observed_file.write(f'"{patient_id}" "{row["Observed_Symptom"]}" {row["Observed_Week"]}\n')
            
            # Écrire dans le fichier History.facts si la condition historique est présente
            if pd.notna(row["History_Condition"]):
                history_file.write(f'"{patient_id}" "{row["History_Condition"]}" {row["History_Count"]}\n')
else:
    print("Les fichiers 'Observed.facts' et 'History.facts' existent déjà.")

Les fichiers 'Observed.facts' et 'History.facts' existent déjà.


### Lecture des fichiers datalog

In [69]:
from pyDatalog import pyDatalog

pyDatalog.clear()

# Charger les faits depuis les fichiers
with open(observed_file_path, "r") as observed_file:
    observed_facts = observed_file.readlines()

with open(history_file_path, "r") as history_file:
    history_facts = history_file.readlines()

for fact in observed_facts:
    # Split each line into its components, remove any leading/trailing whitespace
    fact_parts = fact.strip().split()
    fact_parts = [part.strip('"') for part in fact_parts]
    if fact_parts:
        # pyDatalog.assert_fact('Observed', *fact_parts)
        pyDatalog.assert_fact('Observed', fact_parts[0], fact_parts[1], float(fact_parts[2]))

for fact in history_facts:
    fact_parts = fact.strip().split()
    if fact_parts:
        pyDatalog.assert_fact('History', fact_parts[0], fact_parts[1], float(fact_parts[2]))

print("Faits Observed:")
for fact in observed_facts:
    print(fact.strip())

print("\nFaits History:")
for fact in history_facts:
    print(fact.strip())

Faits Observed:
"1" "depressed_mood" 1.5
"1" "reduced_concentration" 1.2
"1" "reduced_energy" 0.8
"1" "increased_talkativeness" 0.6
"2" "depressed_mood" 5.7
"2" "diminished_interest_pleasure" 5.7
"2" "reduced_concentration" 3.5
"2" "low_self_worth" 2.0
"2" "psychomotor_disturbances" 5.7
"3" "increased_activity_energy" 0.5
"3" "euphoria_irritability_expansiveness" 0.5
"3" "racing_thoughts" 0.5
"3" "increased_talkativeness" 0.5
"3" "increased_self_esteem" 0.5
"3" "diminished_interest_pleasure" 2.0
"3" "reduced_concentration" 2.0
"3" "disrupted_excessive_sleep" 2.0
"3" "change_in_appetite_weight" 2.0
"3" "psychomotor_disturbances" 0.8
"4" "depressed_mood" 5.7
"4" "diminished_interest_pleasure" 5.7
"4" "reduced_concentration" 3.5
"4" "low_self_worth" 2.0
"4" "psychomotor_disturbances" 5.7
"5" "depressed_mood" 7.5
"5" "low_self_worth" 7.5
"5" "disrupted_excessive_sleep" 4.0
"5" "reduced_energy" 5.5
"5" "change_in_appetite_weight" 5.5
"5" "euphoria_irritability_expansiveness" 3.0
"5" "increa

In [72]:
# Définir les relations et les règles en Datalog
pyDatalog.create_terms('Observed, History, Diagnosis, Patient, Symptom, Week, Condition, Count, Disorder')

# Exemple avec un règle très simple
Diagnosis(Patient, Disorder) <= (
    Observed(Patient, 'depressed_mood', Week) & (Week <= 2) & (Disorder == 'BPD2')
)
result = Diagnosis("1", Disorder)
print(result)

Disorder
--------
BPD2    
